In [1]:
from functools import cmp_to_key
import os
import pandas as pd

In [3]:
def compare_file(file1, file2):
    num1 = int(file1.split('.')[0])  
    num2 = int(file2.split('.')[0])  
    if num1 < num2:
        return -1
    elif num1 > num2:
        return 1
    else:
        return 0
def pb_read_appendix1(path="./Problem B/Appendix 1"):
   data=[]
   lower_range=0
   upper_range=2881
   for (_, _, filenames) in os.walk(path):
      temp_save=filenames.copy()
      for idx, ff in enumerate(filenames):
          if ff.startswith("~"):
              temp_save.remove(ff)
      filenames=sorted(temp_save, key=cmp_to_key(compare_file))
      for filename in filenames:
         excel_instance=pd.read_excel(os.path.join(path, filename), sheet_name='Sheet1')
         if len(excel_instance["Var1"].to_list())<(upper_range-lower_range+1):
             continue
         data.append({})
         data[-1]["date"]=[i for i in excel_instance["Var1"].to_list()]
         data[-1]["time"]=[i for i in excel_instance["Var2"].to_list()]
         data[-1]["x"]=[i for i in excel_instance["Var3"].to_list()]
         data[-1]["y"]=[i for i in excel_instance["Var4"].to_list()]
         data[-1]["z"]=[i for i in excel_instance["Var5"].to_list()]
         print(f"{filename}")
   
   for sensor in data:
       sensor["date"]=sensor["date"][lower_range:upper_range]
       sensor["time"]=sensor["time"][lower_range:upper_range]
       sensor["x"]=sensor["x"][lower_range:upper_range]
       sensor["y"]=sensor["y"][lower_range:upper_range]
       sensor["z"]=sensor["z"][lower_range:upper_range]
   return data

In [4]:
appendix1=pb_read_appendix1() #读取所有传感器的数据到一个字典里 只取24:00到4:00的数据！

1.xlsx
2.xlsx
3.xlsx
4.xlsx
5.xlsx
6.xlsx
7.xlsx
8.xlsx
9.xlsx
10.xlsx
11.xlsx
12.xlsx
13.xlsx
14.xlsx
15.xlsx
16.xlsx
17.xlsx
18.xlsx
19.xlsx
20.xlsx
21.xlsx
22.xlsx
23.xlsx
24.xlsx
25.xlsx
26.xlsx
27.xlsx
28.xlsx
29.xlsx
30.xlsx
31.xlsx
32.xlsx
33.xlsx
34.xlsx
35.xlsx
36.xlsx
37.xlsx
38.xlsx
39.xlsx
40.xlsx
41.xlsx
42.xlsx
43.xlsx
44.xlsx
45.xlsx
46.xlsx
47.xlsx
48.xlsx
49.xlsx
50.xlsx
51.xlsx
52.xlsx
53.xlsx
54.xlsx
55.xlsx
56.xlsx
57.xlsx
58.xlsx
59.xlsx
60.xlsx
61.xlsx
62.xlsx
63.xlsx
64.xlsx
65.xlsx
66.xlsx
67.xlsx
68.xlsx
69.xlsx
70.xlsx
71.xlsx
72.xlsx
73.xlsx
74.xlsx
75.xlsx
76.xlsx
77.xlsx
78.xlsx
79.xlsx
80.xlsx
81.xlsx
82.xlsx
83.xlsx
84.xlsx
85.xlsx
86.xlsx
87.xlsx
88.xlsx
89.xlsx
90.xlsx
91.xlsx
92.xlsx
93.xlsx
94.xlsx
95.xlsx
96.xlsx
97.xlsx
98.xlsx
99.xlsx
100.xlsx
101.xlsx
102.xlsx
103.xlsx
104.xlsx
105.xlsx
106.xlsx
107.xlsx
108.xlsx
109.xlsx
110.xlsx
111.xlsx
112.xlsx
113.xlsx
114.xlsx
115.xlsx
116.xlsx
117.xlsx
118.xlsx
119.xlsx
120.xlsx
121.xlsx
122.xlsx
123.xlsx
1

# 读取格式
读取出来是一个array,每个array是每个传感器的信息
## keys 
1. date$\Rightarrow$一个包含不同点日期的array
2. time$\Rightarrow$一个包含对应时间戳的点
3. x$\Rightarrow$该时间戳下的x值
4. y$\Rightarrow$该时间戳下的y值
5. z$\Rightarrow$该时间戳下的z值

In [5]:
import torch 
import numpy as np
device="cuda"

In [6]:
xs=[]
ys=[]
zs=[]
for sensor in appendix1:
    xs.append([x-sensor["x"][0] for x in sensor["x"]])
    ys.append([y-sensor["y"][0] for y in sensor["y"]])
    zs.append([z-sensor["z"][0] for z in sensor["z"]])
xs=np.array(xs)
ys=np.array(ys)
zs=np.array(zs)
xyz=np.stack((xs,ys,zs),axis=2)
print(xs.shape, xyz.shape)
xyz=np.swapaxes(xyz, 0, 1)
xyz=np.swapaxes(xyz, 1, 2)
print(xyz.shape)

(348, 2881) (348, 2881, 3)
(2881, 3, 348)


In [9]:
num_sensors=len(xs)
from mealpy import GA, IntegerVar


def get_best_sensor_by_ga(idx,data:dict):
    def obj_func(sel_idx):
        sel_idx=round(sel_idx[0])
        sel_x=np.array(data["x"][sel_idx])
        sel_y=np.array(data["y"][sel_idx])
        sel_z=np.array(data["z"][sel_idx])
        sel_x=torch.from_numpy(sel_x).to(device)
        sel_y=torch.from_numpy(sel_y).to(device)
        sel_z=torch.from_numpy(sel_z).to(device)
        org_x=torch.from_numpy(data["x"]).to(device)
        org_y=torch.from_numpy(data["y"]).to(device)
        org_z=torch.from_numpy(data["z"]).to(device)
        mse_x=torch.mean(torch.abs(sel_x-org_x)).cpu().numpy()
        mse_y=torch.mean(torch.abs(sel_y-org_y)).cpu().numpy()
        mse_z=torch.mean(torch.abs(sel_z-org_z)).cpu().numpy()
        return (mse_x+mse_y+mse_z)/3
    def obj_func_cpu(sel_idx):
        sel_idx=round(sel_idx[0])
        sel_x=np.array(data["x"][sel_idx])
        sel_y=np.array(data["y"][sel_idx])
        sel_z=np.array(data["z"][sel_idx])
        mse_x=np.mean((data["x"]-sel_x)**2)
        mse_y=np.mean((data["y"]-sel_y)**2)
        mse_z=np.mean((data["z"]-sel_z)**2)
        return (mse_x+mse_y+mse_z)/3
    problem_dict={
        "obj_func":obj_func_cpu,
        "bounds": IntegerVar(lb=[0], ub=[num_sensors-1]),
        "minmax": "min",
    }
    optimizer = GA.BaseGA(epoch=200, pop_size=100, pc=0.85, pm=0.1)
    optimizer.solve(problem_dict)
    print(optimizer.g_best.solution) 
    print(optimizer.g_best.target.fitness) 

In [ ]:
get_best_sensor_by_ga(0, {"x":xyz[3][0][:], "y":xyz[3][1][:], "z":xyz[3][2][:]})
# GPU:7.3s
# CPU:3.1s SO:CPU !

2024/11/29 07:30:02 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: Solving single objective optimization problem.
2024/11/29 07:30:02 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 1, Current best: 0.03788015900371705, Global best: 0.03788015900371705, Runtime: 0.01503 seconds
2024/11/29 07:30:02 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 2, Current best: 0.03788015900371705, Global best: 0.03788015900371705, Runtime: 0.01853 seconds
2024/11/29 07:30:02 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 3, Current best: 0.03788015900371705, Global best: 0.03788015900371705, Runtime: 0.01523 seconds
2024/11/29 07:30:02 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 4, Current best: 0.03788015900371705, Global best: 0.03788015900371705, Runtime: 0.01531 seconds
2024/11/29 07:30:02 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 5, Current best: 0.03788015900371705, Global best: 0.

[209.]
0.03684540804599008


In [29]:
def test_idx(data:dict,sel_idx):
    sel_x=data["x"][sel_idx]
    sel_y=data["y"][sel_idx]
    sel_z=data["z"][sel_idx]
    mse_x=np.mean(np.abs((data["x"]-sel_x)))
    mse_y=np.mean(np.abs((data["y"]-sel_y)))
    mse_z=np.mean(np.abs((data["z"]-sel_z)))
    return (mse_x+mse_y+mse_z)/3
test_idx({"x":xyz[1][0][:], "y":xyz[1][1][:], "z":xyz[1][2][:]},43)

0.127103448275824